In [1]:
# Import Pandas and Numpy
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

# Reading and creating directories
import os

# Ignore warnings
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.filterwarnings(action="ignore", message="credentials were not supplied. open data access only")

In [2]:
# Load data
datafolder = r"C:\Users\alber\Statsbomb"
spadl_h5 = os.path.join(datafolder, "euro2020.h5")

# Load players data
with pd.HDFStore(spadl_h5) as spadlstore:
    
    # Get all players and teams of Euro2020
    players = spadlstore['players']
    teams = spadlstore['teams']
    
    # Get players' minutes and roles for each game and add players' and teams' name
    player_games = spadlstore['player_games'].merge(players, on = 'player_id').merge(teams, on = 'team_id')

In [3]:
# Find total minutes played by each player
player_min = player_games.groupby('player_name').sum()
player_min.reset_index(inplace = True)
player_min = player_min[['player_name','minutes_played']].copy()
player_min = player_min.rename(columns = {'minutes_played':'total_minutes'})

# Filter on minutes played (arbitrary!). Reasoning of 200 = more than 2 games. 
player_min = player_min[player_min.total_minutes > 200 ]

# Aggregate roles on the pitch (arbitrary)
fullbacks = ['Left Back','Right Back','Right Wing Back','Left Wing Back']
centerbacks = ['Center Back','Left Center Back','Right Center Back']
midfielders = ['Center Defensive Midfield','Left Center Midfield','Left Defensive Midfield',
              'Right Center Midfield','Right Defensive Midfield',]
attacking_midfielders = ['Center Attacking Midfield','Left Attacking Midfield','Right Attacking Midfield']
wingers = ['Left Wing','Right Wing','Left Midfield','Right Midfield']
forwards = ['Center Forward','Left Center Forward','Right Center Forward']

# Define a function that aggregates roles from starting positions
def role(row):
    if row['starting_position_name'] == 'Goalkeeper':
        return 'Goalkeeper'
    elif row['starting_position_name'] in fullbacks:
        return 'Fullback'
    elif row['starting_position_name'] in centerbacks:
        return 'Centerback'
    elif row['starting_position_name'] in midfielders:
        return 'Midfielder'
    elif row['starting_position_name'] in attacking_midfielders:
        return 'Attacking Midfielder'
    elif row['starting_position_name'] in wingers:
        return 'Winger'
    elif row['starting_position_name'] in forwards:
        return 'Forward'
    else:
        return 'Substitute'

# Create a column with *role* that specifies the aggregate role for each player
player_games['role'] = player_games.apply(role,axis = 1)

In [4]:
# Filter player_games by minutes played: merge player_min with player_games
player_games = player_min.merge(player_games, on = 'player_name')

# Players started in different positions, hence to link a unique role to each player we decide to find in which role
# he played the most minutes. 
# There could be players that never started,they're roles will be decided based on our experience and
# knowledge about the players.

# Find in which role each player started the most minutes, exclude substitute.
player_roles = (player_games.groupby(['player_name','role','team_name'])['minutes_played'].sum().
                unstack(level = 1).drop('Substitute', axis = 1))
player_roles['most_common_role'] = player_roles.idxmax(axis=1)

In [5]:
# Check if there are players which never started, hence they have only NaN (Substitute column was dropped)
cols_to_check = player_roles.columns
player_roles['is_na'] = player_roles[cols_to_check].isnull().apply(lambda x: all(x), axis=1) 
player_roles[player_roles.is_na == True]

,role,Attacking Midfielder,Centerback,Forward,Fullback,Goalkeeper,Midfielder,Winger,most_common_role,is_na
player_name,team_name,,,,,,,,,
Jordan Brian Henderson,England,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [6]:
# The only player with no role is "Jordan Brian Henderson". Based on our experience is a 'Midfielder'. 
# Being the only one player, handle the missed value 
player_roles['most_common_role'].fillna('Midfielder', inplace = True)

In [7]:
# Check if the missing value is fixed. 
cols_to_check = player_roles.columns
player_roles['is_na'] = player_roles[cols_to_check].isnull().apply(lambda x: all(x), axis=1) 
player_roles[player_roles.is_na == True]

,role,Attacking Midfielder,Centerback,Forward,Fullback,Goalkeeper,Midfielder,Winger,most_common_role,is_na
player_name,team_name,,,,,,,,,


In [8]:
# Reset index and subset the dataframe columns
player_roles.reset_index(inplace = True)
player_roles = player_roles[['player_name','most_common_role','team_name']].copy()

# Merge player_roles and player_min to create a new dataframe 
players_final = player_roles.merge(player_min, on = 'player_name')

# Save dataframe as a csv file
players_final.to_csv('players.csv', index = False)